In [1]:
!pip install ultralytics optuna pyyaml

In [2]:
from ultralytics import YOLO
import optuna
import os
import yaml

In [3]:
# Set this environment variable to disable strict metric checking
os.environ['TUNE_DISABLE_STRICT_METRIC_CHECKING'] = '1'

In [4]:
def train_yolov8(trial):
    model = YOLO("yolov8x.pt")  # Load a pretrained YOLOv8 model

    # Set up the dataset path
    data_path = "/kaggle/input/vehicle-detection/data.yaml"  # Replace with your dataset path
    print(f"Training on dataset: {data_path}")

    # Suggest hyperparameters
    config = {
        "epochs": trial.suggest_int("epochs", 5, 20),
        "batch_size": trial.suggest_categorical("batch_size", [8, 16, 32]),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-3),
        "image_size": trial.suggest_categorical("image_size", [640, 720, 800])
    }

    print(f"Starting training with config: {config}")

    results = model.train(
        data=data_path,
        epochs=config["epochs"],
        batch=config["batch_size"],
        lr0=config["learning_rate"],
        imgsz=config["image_size"]
    )

    print("Training completed, extracting metrics...")

    # Extract validation metrics
    metrics = results.results_dict
    print(f"Validation metrics: {metrics}")

    # Print results per class
    with open(data_path, 'r') as f:
        data_dict = yaml.safe_load(f)
    class_names = data_dict['names']

    # Return the metric to be optimized
    return metrics['metrics/mAP50-95(B)']

In [5]:
def objective(trial):
    return train_yolov8(trial)

In [ ]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=20)

    print("Best trial:")
    trial = study.best_trial
    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    # Train the model with the best hyperparameters
    best_model = YOLO("yolov8x.pt")
    best_results = best_model.train(
        data="/content/drive/MyDrive/dataset/data.yaml",
        epochs=trial.params["epochs"],
        batch=trial.params["batch_size"],
        lr0=trial.params["learning_rate"],
        imgsz=trial.params["image_size"]
    )

    print("\nFinal results with best hyperparameters:")
    print(best_results)

[I 2024-06-24 12:43:16,437] A new study created in memory with name: no-name-b0109c21-83da-4272-8aa6-2a94ddd037b6


Training on dataset: /kaggle/input/vehicle-detection/data.yaml
Starting training with config: {'epochs': 6, 'batch_size': 8, 'learning_rate': 0.0001061599971707366, 'image_size': 720}
Ultralytics YOLOv8.2.41 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/input/vehicle-detection/data.yaml, epochs=6, time=None, patience=100, batch=8, imgsz=720, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=F

/tmp/ipykernel_110/921516691.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-3),
2024-06-24 12:43:17,192	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-24 12:43:18,041	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=15

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytic

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       9/16      11.3G      1.729     0.9184      1.497         60        800: 100%|██████████| 325/325 [05:09<00:00,  1.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:06<00:00,  2.04it/s]

                   all        200       1199       0.53      0.439      0.474      0.233



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/16      11.3G      1.703     0.8788      1.472         41        800:  86%|████████▋ | 281/325 [04:28<00:41,  1.05it/s]